In [15]:
# 模拟两阶段匹配：阶段一匹配偏好类别，阶段二在该类别中匹配 value 值

from sentence_transformers import SentenceTransformer, util
import pandas as pd

# 初始化模型
model = SentenceTransformer('all-MiniLM-L6-v2')
#  这个加载时间也太长了

In [16]:
# 一级分类类别（简洁中性描述）
categories = {
    "pace": "旅行节奏相关偏好",
    "age_group": "旅客年龄段相关偏好",
    "mobility": "行动能力相关偏好",
    "language": "语言沟通相关偏好",
    "avoid": "需要避免的元素",
    "mood": "旅行氛围或体验偏好",
    "attraction": "景点选择相关偏好",
    "food": "饮食选择相关偏好",
    "accommodation": "住宿安排相关偏好",
    "transportation": "交通方式相关偏好"
}
category_embeddings = {
    k: model.encode(v, convert_to_tensor=True)
    for k, v in categories.items()
}

category_embeddings["pace"].shape

/home/homeless/.pyenv/versions/3.10.13/envs/rag/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


torch.Size([384])

In [18]:
# 二级 value 描述（分开写，避免混淆）
value_candidates = {
    "pace": {
        "relaxed": "轻松的旅行节奏，例如每天安排较少、有空闲时间",
        "tight": "紧凑的旅行节奏，例如每天安排密集、活动连贯",
        "free": "自由安排，无严格计划，根据兴趣调整"
    },
    "accommodation": {
        "fixed": "住宿固定在一个地方，不频繁更换酒店",
        "flexible": "可以多次更换住宿地点，根据路线调整",
        "quiet": "住宿环境安静、不嘈杂",
        "convenient": "生活便利，周围有超市、交通便利、设施齐全",
        "luxury": "高端豪华型住宿，如星级酒店、度假村等",
        "cheap": "价格低廉的住宿，适合预算有限的旅客",
        "service": "住宿服务质量好，如清洁、接送、前台服务等"
    },
    "age_group": {
        "kids": "适合带小孩出行的安排，如亲子景点、儿童友好设施等",
        "aged": "适合老年人出行的安排，如缓慢节奏、便利交通等",
        "students": "适合学生群体的安排，如经济实惠、自由灵活",
        "couples": "适合情侣出行的安排，如浪漫景点、私密空间"
    },
    "mobility": {
        "weak": "存在行动不便，如需轮椅、推婴儿车、腿脚不便",
        "not_weak": "行动无明显限制，可自由步行或乘车"
    },
    "language": {
        "barrier": "对语言障碍敏感，偏好中文服务或翻译支持",
        "average": "有一定英文能力，日常交流尚可，或者使用翻译工具无压力",
        "english_speaker": "可以流利使用英语，无语言障碍"
    },
    "avoid": {
        # 不使用二级分类，保持为空
    },
    "mood": {
        "adventure": "追求冒险和刺激体验，如极限运动、探索未知",
        "relax": "偏好放松、疗愈行程，如SPA、海滩、温泉",
        "romantic": "期望浪漫氛围，如情侣旅拍、日落晚餐",
        "cultural": "追求文化沉浸体验，如观展、参加传统活动",
        "spiritual": "寻求心灵平静，如自然冥想、宗教圣地"
    },
    "attraction": {
        "nature": "偏好自然风光类景点，如山川、湖泊、公园",
        "history": "偏好历史文化类景点，如古迹、遗址、博物馆",
        "culture": "偏好城市人文、艺术展览等文化活动",
        "entertainment": "偏好娱乐休闲活动，如游乐园、演出等"
    },
    "food": {
        "feature": "偏好当地特色美食，愿意尝试新口味",
        "no_interest": "对美食兴趣不大，无特别饮食偏好",
        "hybrid": "希望中西结合，选择多样，如既吃中餐也尝西餐"
    },
    "transportation": {
        "public": "偏好公共交通工具，如地铁、公交等",
        "taxi": "偏好出租车或打车类服务，便捷直达",
        "cheap": "希望交通费用低，优先考虑省钱方式",
        "fast": "优先考虑快速到达方式，如高铁、飞机"
    }
}


In [19]:
# 示例用户输入
user_input = "我们带着孩子，不希望太赶，最好轻松一点，也不喜欢天天换地方住，干净一点就好。饮食上怎么方便怎么来，最好是能体验一下当地特色。"


In [20]:

# 文本切块（模拟固定长度分段）
def fixed_chunk(text, max_len=25):
    return [text[i:i+max_len] for i in range(0, len(text), max_len)]
chunks = fixed_chunk(user_input, max_len=10)
chunks
# 所以期待的结果是 节奏轻松，不喜欢换住宿

['我们带着孩子，不希望',
 '太赶，最好轻松一点，',
 '也不喜欢天天换地方住',
 '，干净一点就好。饮食',
 '上怎么方便怎么来，最',
 '好是能体验一下当地特',
 '色。']

In [35]:
import sys
import os

# 手动将当前 notebook 所在目录加入模块路径
sys.path.append(os.path.abspath(".."))
print(os.path.abspath("."))  # 确认当前 notebook 所在路径
os.listdir("..")



/home/homeless/GuideDemo/llm-rag-project/notebooks


['README.md',
 'Pipfile',
 '.git',
 'notebooks',
 'cookies.txt',
 'Dockerfile',
 'images',
 'docker-compose.yaml',
 'data',
 'travel_guide',
 '.env',
 'Pipfile.lock',
 'grafana']

In [41]:
# 使用llm+prompt的方式来给文本添加标点

from travel_guide.rag import llm

def punctuate_and_chunk(text: str, max_len: int = 25) -> list:
    """
    使用 LLM 给文本加标点并切分为句块。
    """
    prompt = f"请仅在下面这段文本中添加或者修改合适的标点符号，返回修改标点后的文本，不要加其他多余的东西：\n{text.strip()}"
    result, _ = llm(prompt)
    punctuated = result.strip()

    # 简单按中文句读切分，句号、问号、感叹号、逗号等
    import re
    raw_chunks = re.split(r'[，。！？\n]', punctuated)
    chunks = [c.strip() for c in raw_chunks if c.strip()]

    # 若仍有长句，可再切固定长度
    final_chunks = []
    for chunk in chunks:
        if len(chunk) <= max_len:
            final_chunks.append(chunk)
        else:
            for i in range(0, len(chunk), max_len):
                final_chunks.append(chunk[i:i + max_len])

    return final_chunks


chunks = punctuate_and_chunk(user_input)
chunks

['我们带着孩子',
 '不希望太急着',
 '最好轻松一点',
 '也不喜欢天天换地方住',
 '干净一点就好',
 '饮食上',
 '怎么方便、怎么来最好是能体验一下当地特色']

In [42]:

# 两阶段匹配
results = []
threshold_cat = 0.5
threshold_val = 0.5


In [47]:

for chunk in chunks:
    chunk_emb = model.encode(chunk, convert_to_tensor=True)
    
    # 阶段一：类别匹配
    cat_scores = {
        cat: float(util.cos_sim(chunk_emb, cat_emb))
        for cat, cat_emb in category_embeddings.items()
    }
    best_cat = max(cat_scores, key=cat_scores.get)
    if cat_scores[best_cat] < threshold_cat:
        continue  # 不足以匹配任何类别
    
    # 阶段二：在该类别下匹配 value 值
    values = value_candidates.get(best_cat, {})
    if not values:
        continue
    val_scores = {
        val: float(util.cos_sim(chunk_emb, model.encode(desc, convert_to_tensor=True)))
        for val, desc in values.items()
    }
    best_val = max(val_scores, key=val_scores.get)
    if val_scores[best_val] < threshold_val:
        continue

    results.append({
        "chunk": chunk,
        "category": best_cat,
        "value": best_val,
        "category_score": round(cat_scores[best_cat], 3),
        "value_score": round(val_scores[best_val], 3)
    })

results

/home/homeless/.pyenv/versions/3.10.13/envs/rag/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[{'chunk': '最好轻松一点',
  'category': 'transportation',
  'value': 'fast',
  'category_score': 0.607,
  'value_score': 0.566},
 {'chunk': '也不喜欢天天换地方住',
  'category': 'accommodation',
  'value': 'fixed',
  'category_score': 0.569,
  'value_score': 0.744},
 {'chunk': '饮食上',
  'category': 'food',
  'value': 'no_interest',
  'category_score': 0.654,
  'value_score': 0.624},
 {'chunk': '怎么方便、怎么来最好是能体验一下当地特色',
  'category': 'transportation',
  'value': 'fast',
  'category_score': 0.659,
  'value_score': 0.663},
 {'chunk': '最好轻松一点',
  'category': 'transportation',
  'value': 'fast',
  'category_score': 0.607,
  'value_score': 0.566},
 {'chunk': '也不喜欢天天换地方住',
  'category': 'accommodation',
  'value': 'fixed',
  'category_score': 0.569,
  'value_score': 0.744},
 {'chunk': '饮食上',
  'category': 'food',
  'value': 'no_interest',
  'category_score': 0.654,
  'value_score': 0.624},
 {'chunk': '怎么方便、怎么来最好是能体验一下当地特色',
  'category': 'transportation',
  'value': 'fast',
  'category_score': 0.659,
  'value_

In [53]:
# 一级分类采用llm
# 二级分类采用文本分类
def match_preferences(text: str, threshold_val=0.5, max_chunk_len=25) -> list:
    """
    对用户输入文本进行处理：
    1. 加标点
    2. 分句
    3. LLM 判断每个句子归属的字段（一级分类）
    4. 使用 embedding 匹配具体值（value）
    返回：匹配结果列表
    """
    chunks = punctuate_and_chunk(text, max_len=max_chunk_len)
    
    results = []

    for chunk in chunks:
        # 1. 调用 LLM 判断一级分类
        prompt = f"请从以下字段中选择这句话最相关的一个：\n\n{chunk}\n\n字段选项：['pace', 'accommodation', 'food', 'transportation', 'age_group', 'mobility', 'language', 'avoid', 'mood', 'attraction']\n\n仅返回字段名称，例如：'pace'"
        field_result, _ = llm(prompt)
        print(chunk)
        print(field_result)
        best_cat = field_result.strip().lower()

        value_map = value_candidates.get(best_cat, {})
        if not value_map:
            continue

        # 2. 使用 embedding 匹配字段下的值
        chunk_emb = model.encode(chunk, convert_to_tensor=True)
        val_scores = {
            val: float(util.cos_sim(chunk_emb, model.encode(desc, convert_to_tensor=True)))
            for val, desc in value_map.items() if desc.strip()
        }
        if not val_scores:
            continue

        best_val = max(val_scores, key=val_scores.get)
        print(best_val)
        if val_scores[best_val] < threshold_val:
            continue

        results.append({
            "chunk": chunk,
            "category": best_cat,
            "value": best_val,
            "value_score": round(val_scores[best_val], 3)
        })

    return results

match_preferences(user_input)

我们带着孩子
'mobility'
不希望太赶
pace
tight
最好轻松一点
'pace'
也不喜欢天天换地方住
'accommodation'
干净一点就好
'mood'
饮食上
'food'
怎样方便怎样来
transportation
fast
最好能体验一下当地特色
'food'


[{'chunk': '不希望太赶', 'category': 'pace', 'value': 'tight', 'value_score': 0.55},
 {'chunk': '怎样方便怎样来',
  'category': 'transportation',
  'value': 'fast',
  'value_score': 0.72}]

In [ ]:
# 上面那个相似性的方法不太行
from travel_guide.generate_prompt import classify_preferences_llm

classify_preferences_llm(user_input)
# 全使用llm更是扯淡

{'pace': '',
 'age_group': '',
 'mobility': '',
 'language': '',
 'avoid': '',
 'mood': '',
 'attraction': '',
 'food': '',
 'accommodation': '',
 'transportation': ''}

         [用户原文]
              ↓
     +--------+--------+--------+
     |        |        |        |
 [pace]   [accommodation]   [food]
   ↓             ↓            ↓
value1,2,3   value1,2,3   value1,2,3
(sim match)  (sim match)  (sim match)
   ↓             ↓            ↓
"relaxed"    "fixed"      "feature"

采用这种方案，schema.py先不要删除一级分类的部分


In [9]:
# 新的schema
new_schema = {
    "pace": {
        "relaxed": [
            "我们想轻松一点，不要太赶",
            "每天行程少一些比较好",
            "能慢慢体验就行了"
        ],
        "tight": [
            "希望行程安排紧凑一点",
            "多安排些活动",
            "尽量不要浪费时间",
            "希望一次将这里的景点都看完"
        ],
        "free": [
            "我们想自由一些，不按固定计划走",
            "看当天心情安排活动",
            "临时决定去哪都可以",
            "随便"
        ]
    },
    "accommodation": {
        "fixed": [
            "不想天天换地方住",
            "住宿最好固定一个地方",
            "希望住在同一个酒店",
            "不想到处乱跑",
            "选一个好地方一直住着",
            "不像整天搬行李",
            "找个地方待着就行"
        ],
        "flexible": [
            "可以根据路线调整住宿",
            "愿意换不同地方住住看",
            "根据景点调整住所"
        ],
        "quiet": [
            "希望住宿环境安静",
            "不要太吵的地方"
        ]
    },
    "food": {
        "feature": [
            "想尝试当地特色美食",
            "想体验本地的饮食文化"
        ],
        "no_interest": [
            "吃什么无所谓",
            "对吃的没特别要求",
            "怎么方便怎么来",
            "饿不死就行",
            "不饿着就行",
            "吃泡面也行",
            "其实便利店也可以，吃泡面也可以接受"
        ],
        "hybrid": [
            "中餐西餐都可以",
            "希望饮食选择多样化"
        ]
    }
}


In [6]:
from sentence_transformers import SentenceTransformer, util
from collections import defaultdict

model = SentenceTransformer('all-MiniLM-L6-v2')


def retrieve_preferences(text: str, schema: dict, threshold=0.6) -> dict:
    """
    使用语义匹配方式，从传入的表达样例 schema 中检索最相似的偏好值。
    返回结构化偏好字段。
    """
    preferences = {field: "" for field in schema}
    query_emb = model.encode(text, convert_to_tensor=True)

    for field, value_map in schema.items():
        best_score = 0
        best_value = ""
        for val, example_list in value_map.items():
            for example in example_list:
                example_emb = model.encode(example, convert_to_tensor=True)
                score = float(util.cos_sim(query_emb, example_emb))
                if score > best_score:
                    best_score = score
                    best_value = val
        if best_score >= threshold:
            preferences[field] = best_value

    return preferences

In [10]:
from pprint import pprint

user_input = "我们想轻松一点，不要太赶，也不想天天换地方住。吃饭上能方便就行，也希望能体验当地特色美食。"
result = retrieve_preferences(user_input, schema=new_schema)
pprint(result)

/home/homeless/.pyenv/versions/3.10.13/envs/rag/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'accommodation': 'fixed', 'food': 'feature', 'pace': 'relaxed'}


In [12]:
user_input = "我不想到处乱跑，找个地方清净一直待着就行，吃上不讲究，不饿着就行"
result = retrieve_preferences(user_input, schema=new_schema)
pprint(result)


{'accommodation': 'fixed', 'food': 'no_interest', 'pace': 'free'}
